In [ ]:
# Get access to my files

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

# Import file downloaded from Google BigQuery
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Postdoc/bq-results-20250711-144732-1752245533070.csv")

In [ ]:
# Column "subject_id" is the patient number, "hadm_id" is the hospital admission number, "note" is the discharge summary,
# and "label" is a custom column where value 1 means the ICD-9 code is between 290 and 319 (psychiatric)

print(df.head())        # shows first 5 rows
print(df.columns)       # shows all column names
print(df.shape)         # (rows, columns)

In [ ]:
# Return all psychiatric cases, there are 17448 of them out of 59652 entries

df.loc[df['label'] == 1]

In [ ]:
import re

# Function to clean the note column by removing clinician signature, placeholders, and empty space
def clean_note(note):
    # Lowercase for consistency
    note = note.lower()

    # Remove clinician signature lines
    note = re.sub(r"(?i)(dictated|electronically signed|attending).*", "", note)

    # Remove PHI placeholder tags like [**Name**] or [**Date**]
    note = re.sub(r"\[\*\*.*?\*\*\]", " ", note)

    # Remove excess whitespace and line breaks
    note = re.sub(r"\s+", " ", note).strip()

    return note

In [ ]:
# Function to extract a particular section in the note column
def extract_section(text, section_name):
    pattern = re.compile(rf"{section_name}:(.*?)(\n[A-Z][A-Z\s]+:|\Z)", re.DOTALL | re.IGNORECASE)
    match = pattern.search(text)
    return match.group(1).strip() if match else ""

In [ ]:
# Extract "history of present illness"
df['hpi'] = df['note'].apply(lambda x: extract_section(x, "history of present illness"))

In [ ]:
# Extract "past medical history"
df['pmh'] = df['note'].apply(lambda x: extract_section(x, "past medical history"))

In [ ]:
# Extract "social history"
df['sh'] = df['note'].apply(lambda x: extract_section(x, "social history"))

In [ ]:
# Extract "family history"
df['fh'] = df['note'].apply(lambda x: extract_section(x, "family history"))

In [ ]:
# Combine the extracted sections of "note" into the new column "relevant_note"
df['relevant_note'] = df['hpi'] + ' ' + df['pmh'] + ' ' + df['sh'] + ' ' + df['fh']

In [ ]:
# Clean the "relevant_note" column
df['clean_relevant_note'] = df['relevant_note'].apply(clean_note)

In [ ]:
# Truncate "clean_relevant_note" according to tokenizer (512 tokens max)

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

# Add a column with number of tokens (optional)
df['token_count'] = df['clean_relevant_note'].apply(lambda x: len(tokenizer.tokenize(x)))

# Truncate overly long notes
df['clean_relevant_note_truncate'] = df['clean_relevant_note'].apply(lambda x: " ".join(x.split()[:512]))

In [ ]:
# Remove entries with empty "clean_relevant_note_truncate" columns

df_strip = df[df["clean_relevant_note_truncate"].str.strip().astype(bool)]
print(df.shape)
print(df_strip.shape)

# Reduced total number of entries from 59652 to 51695

(59652, 12)
(51695, 12)


In [ ]:
# Only save the essential columns into a csv file

df_strip[['subject_id', 'hadm_id', 'label', 'clean_relevant_note_truncate']].to_csv("/content/drive/MyDrive/Colab Notebooks/Postdoc/cleaned_stripped_mimic_notes.csv", index=False)